In [1]:
### Imports
from pathlib import Path
import scipy.sparse as sparse
import implicit
import pandas as pd
import numpy as np
import operator
import ast
from scipy.spatial.distance import cosine
import collections
import heapq

In [2]:
# Functions

def make_prior_data():
    # Read prior order csv
    df_order_products_prior = pd.read_csv("../data/order_products__prior.csv")
    current_order_user_df = df_orders.loc[(df_orders.eval_set == "prior")].reset_index()
    current_order_user_df = current_order_user_df[["order_id", "user_id"]]

    assert len(current_order_user_df["order_id"].unique()) == len(df_order_products_prior["order_id"].unique())

    # Group product_id for each order into products
    df_order_products_prior = df_order_products_prior[["order_id", "product_id"]]
    df_product_frequency = df_order_products_prior['product_id'].value_counts()
    df_order_products_prior = df_order_products_prior.groupby("order_id")["product_id"].apply(list).reset_index().rename(columns={"product_id": "products"})
    
    
    assert current_order_user_df.size == df_order_products_prior.size

    df_prior_user_products = pd.merge(current_order_user_df, df_order_products_prior, on="order_id")
    df_prior_user_products = df_prior_user_products[["user_id", "products"]]
    df_prior_user_products = df_prior_user_products.groupby("user_id")["products"].agg(sum).reset_index()

    return df_prior_user_products, df_product_frequency


## Load datasets

In [3]:
# Order datasets
df_orders = pd.read_csv("../data/orders.csv")
df_aisles = pd.read_csv("../data/aisles.csv")
df_departments = pd.read_csv("../data/departments.csv")
df_products = pd.read_csv("../data/products.csv")

In [4]:
# Running time: 3 min
df_prior_user_products, df_product_frequency = make_prior_data()

In [6]:
df_prior_user_products.head()

,user_id,products
0,1,"[196, 14084, 12427, 26088, 26405, 196, 10258, ..."
1,2,"[32792, 47766, 20574, 12000, 48110, 22474, 165..."
2,3,"[9387, 17668, 15143, 16797, 39190, 47766, 2190..."
3,4,"[36606, 7350, 35469, 2707, 42329, 7160, 1200, ..."
4,5,"[15349, 21413, 48775, 28289, 8518, 11777, 3171..."


In [7]:
df_product_frequency = pd.DataFrame(df_product_frequency).rename(columns={"product_id": "frequency"})
df_product_frequency.head()

,frequency
24852,472565
13176,379450
21137,264683
21903,241921
47209,213584


In [14]:
df_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [44]:
df_aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [16]:
df_departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [17]:
df_products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


## Clean data

In [ ]:
# Consider only "prior" orders and remove all columns except order_id and user_id from df_orders
prior_order_user = df_orders.loc[df_orders.eval_set == "prior"]
prior_order_user = prior_order_user[["order_id", "user_id"]]

In [ ]:
prior_order_user.head()

In [ ]:
# Remove all columns except order_id and user_id from df_orders
prior_order_product = df_order_products_prior[["order_id", "product_id"]]

In [ ]:
prior_order_product.head()

## Build product_hourCount Dictionary

In [36]:
product_hourCount = {}

times = 0
for index, oid in enumerate(df_orders.order_id):
    selected_order_table = df_order_products_prior.loc[df_order_products_prior['order_id'] == oid]
    for pid in selected_order_table['product_id']:
        if pid not in product_hourCount:
            hour_count = {}
            hour_count[df_orders.order_hour_of_day[index]] = 1
            product_hourCount[pid] = hour_count
        else:
            hour_count = product_hourCount[pid]
            hour = df_orders.order_hour_of_day[index]
            hour_count[hour] = hour_count.get(hour, 0) + 1
    times += 1
    if times > 100:
        break

frequent_order_hour_of_day = {}
for product_id in product_hourCount:
    frequent_order_hour_of_day[product_id] = max(product_hourCount[product_id].items(), key=operator.itemgetter(1))[0]
    # key: product, value: frequent order hour of day

frequent_hour_department = {}
for product_id, hour in frequent_order_hour_of_day.items():
    department = df_products.loc[df_products['product_id'] == product_id].department_id.item()
    frequent_hour_department[hour] = frequent_hour_department.get(hour, []) + [department]
for hour in frequent_hour_department:
    print(hour, ": ", sorted(frequent_hour_department[hour]))


7 :  [7, 14, 16, 19, 19]
8 :  [1, 4, 4, 4, 9, 12, 13, 15, 16, 16, 16, 16, 16, 16, 16, 17, 19, 19, 19, 19]
9 :  [1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 9, 11, 11, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 17, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 21]
12 :  [3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 6, 7, 7, 7, 13, 13, 13, 14, 14, 16, 16, 16, 16, 16, 16, 16, 16, 19, 19, 19, 19, 19, 20, 20, 20]
15 :  [1, 1, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 6, 7, 7, 9, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 15, 16, 16, 16, 16, 19, 19, 20, 20, 20, 20, 20]
14 :  [3, 4, 4, 4, 4, 4, 7, 7, 7, 16, 16, 16, 16, 16, 19, 19]
10 :  [1, 1, 1, 1, 3, 4, 4, 4, 4, 4, 6, 7, 7, 7, 7, 13, 16, 16, 16, 16, 16, 16, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20]
16 :  [1, 4, 4, 4, 4, 4, 4, 4, 4,

## Build tf-idf

In [8]:
# Total users (rows/documents) in the utility matrix
print('Total users: {}'.format(len(df_prior_user_products)))

Total users: 206209


In [9]:
# Calculate term (item) frequency

# Running time: ~20 min for the first 100,000 rows
tf1 = df_prior_user_products[:100000].products.apply(pd.value_counts).fillna(0)

In [ ]:
# Running time: N/A
tf2 = df_prior_user_products[100000:].products.apply(pd.value_counts).fillna(0)

In [10]:
# tf = pd.concat([tf1, tf2], ignore_index=True)
tf = tf1
tf.head()

,1,2,3,4,5,6,7,8,9,10,...,49679,49680,49681,49682,49683,49684,49685,49686,49687,49688
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Calculate Inverse Document Frequency
# Running time: (estimated) 10 - 15 min for all, 5 min for the first half 100,000 rows
# using simple smoothing
# gt(num) = greater than num
idf = np.log((len(df_prior_user_products) + 1 ) / (tf.gt(0).sum() + 1))

In [12]:
print(idf)

1         6.353328
2         8.386503
3         8.498981
4         7.736841
5        11.543503
6        11.138038
7         9.528600
8         8.452461
9         8.573089
10        5.768952
11        9.058597
12        8.125776
13       10.444891
14       10.627212
15       11.543503
16       10.157209
17       10.290740
18        8.710290
19       11.138038
20       11.543503
21       10.627212
22        9.671701
23        6.601861
24       10.850356
25        6.347772
26        8.523078
27        8.077767
28        7.049265
29        7.919162
30        9.145608
           ...    
49658     9.528600
49659     7.736841
49661    11.543503
49662    10.157209
49663    10.627212
49664     8.266358
49665     8.802663
49666    11.138038
49667     6.373019
49668     7.725791
49669    11.138038
49670     8.176207
49671     9.464062
49672    10.444891
49673    10.039426
49674     9.838755
49675     9.240918
49676     9.528600
49677     7.959984
49678     7.631480
49679     8.344830
49680     6.

In [13]:
# Calculate tf_idf

tf_idf = tf * idf
tf_idf.head()

,1,2,3,4,5,6,7,8,9,10,...,49679,49680,49681,49682,49683,49684,49685,49686,49687,49688
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.640368,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [14]:
# Calculate Cosine Distance
# Running time: (estimated) 16 - 20 min for all, 8 min for the first half 100,000 rows
def fetchMostSimilarUserTo(target_user_id, length, tf_idf):
    target_user = tf_idf.loc[target_user_id - 1]
    cos_vec = tf_idf.apply(lambda other_user: 1 - cosine(target_user, other_user), axis=1).fillna(0)
    return cos_vec

# Change target_user_id HERE.
target_user_id = 1
cos_vec = fetchMostSimilarUserTo(target_user_id, len(tf_idf), tf_idf)

In [15]:
# Select top 10 similar users
top_10_similar_uers = cos_vec.nlargest(10)
# Format
# user_id - 1  |  Cosine Distance
print(top_10_similar_uers)

0        1.000000
38486    0.708020
61532    0.560269
63730    0.540606
39852    0.529910
10398    0.527607
48738    0.492225
59666    0.478180
98549    0.475943
13390    0.474704
dtype: float64


In [17]:
# Exclude the user with same purchase history (1.00000) as the target user and implement set-minus
products_target_user = df_prior_user_products.loc[df_prior_user_products['user_id'] == target_user_id].products

# Initialize the result for recommendations
recommendations = []

# Products of Target User
productset_target_user = set(products_target_user.tolist()[0])

# Fetch the preliminary recommendations
for i, similarity in top_10_similar_uers.iteritems():
    
    products_similar_user = df_prior_user_products.loc[df_prior_user_products['user_id'] == i + 1].products
    
    # Recommend the products bought by the user who firstly differs in the purchase history from A.
    candidate_recommendation = set(products_similar_user.tolist()[0]) - set(products_target_user.tolist()[0])
    
    # If similarity equals to 1 (namely same purchase history) or the candidate_recommendation is empty,
    # skip current user
    if similarity // 1 == 1 or not candidate_recommendation: continue
    
    # One candidate_recommendation found, and extend it to the result
    recommendations.extend(candidate_recommendation)

    # If length of recommendations exceed 10, break
    if len(recommendations) > 10: break

In [18]:
# Pick the top 10 popularity (overall sales) to recommend
h = []
for rec in recommendations:
    heapq.heappush(h, (df_product_frequency.loc[rec, 'frequency'], rec))
    if len(h) > 10:
        heapq.heappop(h)

In [23]:
# Output the product_name of Target User's products as well as Recommendations
print('Actual products bought by User {}:'.format(target_user_id))
print([df_products.iloc[product_id]['product_name'] for product_id in productset_target_user])
print()
print('Recommended products for User {}:'.format(target_user_id))
print([df_products.iloc[item[1]]['product_name'] for item in h])

Actual products bought by User 1:
['Organic Honeydew', 'Cold Brew Coffee Tahitian Vanilla', "Spot's Pate Cat Grain Free Ground Whitefish", 'Epic Fruit & Yogurt Filled Pouches', 'Beet Kombucha', 'Broccoli Squash Carrots Onion Red Pepper Steamables', 'Grape Nut Flakes Cereal', 'Triple Distilled Irish Whiskey', "Steam'ables Green Peas", 'Cilantro Bunch', 'Peachtree Schnapps', 'Original Pretzel Crisps', 'Chocolate Caramel Pudding Snack Pack', 'Pasta & Enchilada Sauce, Organic, 7 Veggie', '80% Lean Ground Beef', 'Creamy Chicken & Shrimp in a Parmesan Alfredo Sauce', 'Warrior Blend Vanilla Dietary Supplement', 'Organic Creamy Cashewmilk']

Recommended products for User 1:
['Brioche Bachelor Loaf', 'Petit Suisse Fruit', 'Premium Genoa', 'Original Antacid & Pain Relief Tablets 36 Ct', 'Maximum Strength Anti-Itch Creme', 'CleanWear Ultra Thin Regular with Wings Pads', 'Valdosta Pecans With Cranberries, Black Pepper & Orange Zest', 'Sharpened No. 2 Pencils', 'Traditional Refried Pinto Beans', 'G

# Playground

In [26]:
df1 = pd.DataFrame([
        [1, 196],
        [1, 196],
        [1, 134],
        [1, 196],
        [1, 134]
    ], columns=['order_id', 'product_id'])

df1.head()

,order_id,product_id
0,1,196
1,1,196
2,1,134
3,1,196
4,1,134


In [27]:
df2 = pd.DataFrame([
        [2, 196],
        [2, 196],
        [2, 134],
        [2, 196],
        [2, 134]
    ], columns=['order_id', 'product_id'])

df2.head()

,order_id,product_id
0,2,196
1,2,196
2,2,134
3,2,196
4,2,134


In [30]:
df = pd.concat([df1, df2], ignore_index=True)

In [31]:
df['product_id'].value_counts()

196    6
134    4
Name: product_id, dtype: int64

In [5]:
df_products.iloc[0]['product_name']

'Chocolate Sandwich Cookies'